In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
import tensorflow as tf
import tensorflow_hub as hub

c:\Users\ainao\anaconda3\envs\sentA\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [6]:
import tensorflow as tf
import tensorflow_hub as hub




In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: Path
    all_params: dict

In [8]:
from sentimentanalyzer.utils.common import read_yaml, create_directories  # adjust import as needed
from sentimentanalyzer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/model_trainer_USE/model.h5",
            test_data="artifacts/model_trainer_USE/test.tfrecord",
            all_params=self.params
        )
        return eval_config

In [10]:
import tensorflow as tf
import json
from pathlib import Path
from sentimentanalyzer.utils.common import save_json

In [11]:
class Evaluation:
    def __init__(self, config: 'EvaluationConfig'):
        self.config = config
        self.model = None
        self.test_ds = None
        self.score = None
       

    def load_model(self) -> tf.keras.Model:
        """
        Load and return the Keras model from the configured path.
        """
        model_path: Path = self.config.path_of_model
        self.model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})
        return self.model


    def load_test_dataset(self, batch_size: int = 32, tfrecord_path: str = None):
       
       """
       Load the test split from TFRecord—deterministic
       """
       # Determine file path
       path = tfrecord_path or os.path.join(self.config.test_data)
   
       # Define feature spec once
       feature_spec = {
        'text':   tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.int64),
       }

       # Parser function (no lambda)
       def _parse_example(serialized):
           parsed = tf.io.parse_single_example(serialized, feature_spec)
           return parsed['text'], parsed['target']

       # Build dataset
       ds = (
            tf.data.TFRecordDataset(path)
            .map(_parse_example, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

       self.test_ds = ds
       return ds



    def evaluate(self) -> tuple:
         self.load_model()
         self.test_ds = self.load_test_dataset()
         self.score = self.model.evaluate(self.test_ds)
         self.save_score()
         return self.score

       

    def save_score(self, output_path: Path = Path("scores.json")) -> None:
        """
        Save evaluation results to a JSON file at output_path.
        """
        if self.score is None:
            raise ValueError("No evaluation score to save. Run evaluate() first.")

        loss, accuracy = self.score
        scores = {"loss": float(loss), "accuracy": float(accuracy)}
        save_json(path=output_path, data=scores)


In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation = Evaluation(val_config)
    evaluation.load_model()
    evaluation.load_test_dataset()
    evaluation.evaluate()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-06-15 11:26:45,987: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-15 11:26:45,997: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-15 11:26:46,003: INFO: common: created directory at: artifacts]
[2025-06-15 11:26:46,090: INFO: resolver: Using C:\Users\ainao\AppData\Local\Temp\tfhub_modules to cache modules.]
[2025-06-15 11:26:52,073: INFO: load: Fingerprint not found. Saved model loading will continue.]
[2025-06-15 11:27:00,625: INFO: load: Fingerprint not found. Saved model loading will continue.]


c:\Users\ainao\anaconda3\envs\sentA\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


32/32 [==============================] - 2s 49ms/step - loss: 0.5905 - sparse_categorical_accuracy: 0.6800
[2025-06-15 11:27:07,255: INFO: common: json file saved at: scores.json]
[2025-06-15 11:27:07,259: INFO: common: json file saved at: scores.json]
